In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
import google.generativeai as genai
import json
from pinecone import Pinecone, ServerlessSpec


/Users/lilyanhakim/anaconda3/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",
    dimension= 768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [3]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'John Smith',
  'subject': 'Physics',
  'stars': 5,
  'review': 'The professor was engaging and made the material easy to understand.'},
 {'professor': 'Emily Johnson',
  'subject': 'Economics',
  'stars': 4,
  'review': 'Challenging course, but the professor was very knowledgeable.'},
 {'professor': 'Michael Brown',
  'subject': 'Chemistry',
  'stars': 3,
  'review': "The professor's lectures were very informative."},
 {'professor': 'Sarah Davis',
  'subject': 'History',
  'stars': 5,
  'review': 'Great teaching style, very interactive.'},
 {'professor': 'David Taylor',
  'subject': 'Biology',
  'stars': 2,
  'review': 'Material was difficult to follow at times.'},
 {'professor': 'Jessica White',
  'subject': 'Philosophy',
  'stars': 4,
  'review': 'Very thought-provoking and engaging lectures.'},
 {'professor': 'James Martinez',
  'subject': 'Sociology',
  'stars': 5,
  'review': 'Professor made the subject very interesting.'},
 {'professor': 'Laura Anderson',
  'subje

In [10]:
processed_data = []
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

for review in data["reviews"]:
    response = genai.embed_content(
        content=review['review'], 
        model="models/text-embedding-004"
    )
    embedding = response.get('embedding', None)

    if embedding is not None:
        processed_data.append(
          {
            "values": embedding,
            "id": review.get('professor') or review.get('tutor') or review.get('counselor'),
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )
    else:
        print("Error: 'embedding' key not found in response")
    

I0000 00:00:1724571870.709610 9920944 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [11]:
index = pc.Index('rag')
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

Upserted count: 45


In [14]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 28}},
 'total_vector_count': 28}